In [ ]:
# Install necessary libraries
!pip install transformers diffusers accelerate opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import StableDiffusionPipeline
import cv2
import numpy as np
from PIL import Image
import os
from IPython.display import display, Image as IPImage

In [ ]:
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

scheduler%2Fscheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

tokenizer%2Fvocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

vae%2Fconfig.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# Function to generate image from a text prompt
def generate_image(prompt, num_inference_steps=50, seed=None):
    generator = torch.manual_seed(seed) if seed is not None else None
    image = pipe(prompt, num_inference_steps=num_inference_steps, generator=generator).images[0]
    return image

In [ ]:
# Function to create video from generated frames (with 1 frame per second)
def create_video_from_frames(frame_folder, output_video_path, frame_rate=1):  # Set frame_rate to 1
    frame_files = sorted([f for f in os.listdir(frame_folder) if f.endswith('.png')])
    frame_array = []

    for filename in frame_files:
        img_path = os.path.join(frame_folder, filename)
        frame = cv2.imread(img_path)
        frame_array.append(frame)

    height, width, _ = frame_array[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # For .mp4 file
    out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    for frame in frame_array:
        out.write(frame)

    out.release()

In [ ]:
# Set up a folder for generated frames
frame_folder = "/content/frames"
os.makedirs(frame_folder, exist_ok=True)

In [ ]:
# Generate frames for the animation (e.g., 10 frames)
num_frames = 10
prompt = "a futuristic city skyline at sunset, animated style"
for i in range(num_frames):
    print(f"Generating frame {i+1}/{num_frames}")
    img = generate_image(prompt, seed=i)  # Changing the seed to get slightly different images
    img.save(f"{frame_folder}/frame_{i+1}.png")

Generating frame 1/10


  0%|          | 0/50 [00:00<?, ?it/s]

Generating frame 2/10


  0%|          | 0/50 [00:00<?, ?it/s]

Generating frame 3/10


...

  0%|          | 0/50 [00:00<?, ?it/s]

Generating frame 9/10


  0%|          | 0/50 [00:00<?, ?it/s]

Generating frame 10/10


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Create a video from the generated frames with 1 frame per second (10 seconds total)
output_video_path = "/content/animation.mp4"
create_video_from_frames(frame_folder, output_video_path, frame_rate=1)

In [ ]:
# Display the generated video
from IPython.display import Video
Video(output_video_path)